In [1]:
data_path = '/Users/allaingu/Data/chess_id'
train_data_path = '/Users/allaingu/Data/chess_id/train'

! ls $train_data_path

bb    bk    bn    bp    bq    br    empty wb    wk    wn    wp    wq    wr


In [2]:
import torch
from torchvision import datasets
from torchvision import transforms

# https://discuss.pytorch.org/t/how-to-preprocess-input-for-pre-trained-networks/683/9
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225])

train_dataset = datasets.folder.ImageFolder(
    train_data_path,
    transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        normalize,
    ])
)

In [3]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=1, shuffle=True)

train_loader

image_tensor, label  = next(iter(train_loader))
print(image_tensor.shape)
image_tensor, label

torch.Size([1, 3, 256, 256])


(
 ( 0 , 0 ,.,.) = 
   0.5707  0.5536  0.6392  ...   0.0741 -0.1314 -0.0972
  -0.6794 -0.6965 -0.6623  ...   0.6221  0.7591  0.9303
  -1.0904 -1.1247 -1.1760  ...   0.7248  1.1358  1.3070
            ...             ⋱             ...          
  -1.5528 -1.5014 -1.4672  ...   0.9132  1.3413  1.3927
  -0.9877 -0.9877 -0.8507  ...   0.8104  1.2728  1.3413
   0.4166  0.5707  0.5364  ...   0.5878  1.0673  1.1529
 
 ( 0 , 1 ,.,.) = 
   0.6429  0.6604  0.8004  ...   0.3277  0.0476  0.0301
  -0.5826 -0.5826 -0.4776  ...   0.8880  0.9755  1.0805
  -0.8978 -0.8978 -0.8978  ...   0.9580  1.3431  1.4657
            ...             ⋱             ...          
  -1.1429 -1.0903 -1.0378  ...   1.0280  1.4657  1.5182
  -0.6877 -0.6527 -0.5126  ...   0.9230  1.3957  1.4657
   0.6254  0.7829  0.7479  ...   0.6954  1.1856  1.2731
 
 ( 0 , 2 ,.,.) = 
   0.2173  0.2173  0.3742  ...  -0.0964 -0.3404 -0.3753
  -0.9504 -0.9504 -0.8633  ...   0.4614  0.5834  0.6879
  -1.2293 -1.2293 -1.2119  ...   0.5659  0.9

In [4]:
train_dataset.classes

['bb',
 'bk',
 'bn',
 'bp',
 'bq',
 'br',
 'empty',
 'wb',
 'wk',
 'wn',
 'wp',
 'wq',
 'wr']

In [5]:
from torchvision import models
from torch import nn

model = models.alexnet(pretrained=True)

In [6]:
model.features

Sequential (
  (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU (inplace)
  (2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU (inplace)
  (5): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU (inplace)
  (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU (inplace)
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU (inplace)
  (12): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
)

In [7]:
from torch.autograd import Variable

image_features = model.features(Variable(image_tensor))
print(image_features.data.shape)

print(image_features.view(image_features.size(0), 256 * 7 * 7).data.shape)

torch.Size([1, 256, 7, 7])
torch.Size([1, 12544])


In [34]:
from torch import nn


class ImageClassifier(nn.Module):
    LAST_LAYER_SIZE = 256 * 7 * 7
    
    def __init__(self, num_classes, alexnet_model):
        super(ImageClassifier, self).__init__()
        self.feature_model = alexnet_model.features
        # Freeze those weights
        for p in self.feature_model.parameters():
            p.requires_grad = False

        self.classifier = nn.Sequential(
            nn.Linear(self.LAST_LAYER_SIZE, num_classes)
        )
        
    def forward(self, x):
        features = self.feature_model(x)
        flattened_features = features.view(features.size(0), -1)
        return self.classifier(flattened_features)

    
class ImageClassifier(nn.Module):
    LAST_LAYER_SIZE = 256 * 7 * 7
    
    def __init__(self, num_classes, alexnet_model):
        super(ImageClassifier, self).__init__()
        self.feature_model = alexnet_model.features
        # Freeze those weights
        for p in self.feature_model.parameters():
            p.requires_grad = False

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(self.LAST_LAYER_SIZE, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, num_classes),
        )
        
    def forward(self, x):
        features = self.feature_model(x)
        flattened_features = features.view(features.size(0), -1)
        return self.classifier(flattened_features)

In [23]:
classifier = ImageClassifier(num_classes=len(train_dataset.classes), alexnet_model=models.alexnet(pretrained=True))

predictions = classifier(Variable(image_tensor))

predictions

Variable containing:

Columns 0 to 9 
-0.7115 -0.7850 -0.7554 -0.2374  0.0073 -0.1187 -0.5541 -0.0968 -0.0438  0.9178

Columns 10 to 12 
 0.4998 -0.0048 -0.0571
[torch.FloatTensor of size 1x13]

In [10]:
import operator

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32, shuffle=True)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, classifier.parameters()), lr=0.01,
                            momentum=0.9,
                            weight_decay=1e-4)

for i, (input, target) in enumerate(train_loader):
    input_var = torch.autograd.Variable(input)
    target_var = torch.autograd.Variable(target)

    # compute output
    output = classifier(input_var)
    loss = criterion(output, target_var)
    if i % 10 == 0:
        print(loss.data[0])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


 2.7316
[torch.FloatTensor of size 1]


 3.9307
[torch.FloatTensor of size 1]


 3.2967
[torch.FloatTensor of size 1]


 2.6984
[torch.FloatTensor of size 1]


 1.3197
[torch.FloatTensor of size 1]


 0.9563
[torch.FloatTensor of size 1]


 3.3618
[torch.FloatTensor of size 1]


 4.1733
[torch.FloatTensor of size 1]


 1.4293
[torch.FloatTensor of size 1]


 3.5195
[torch.FloatTensor of size 1]


 1.5722
[torch.FloatTensor of size 1]


 1.6089
[torch.FloatTensor of size 1]


 2.5957
[torch.FloatTensor of size 1]


 1.8126
[torch.FloatTensor of size 1]


 1.3293
[torch.FloatTensor of size 1]


 3.1214
[torch.FloatTensor of size 1]


 2.8359
[torch.FloatTensor of size 1]


 0.8738
[torch.FloatTensor of size 1]


 0.8968
[torch.FloatTensor of size 1]


 1.1159
[torch.FloatTensor of size 1]


 0.6859
[torch.FloatTensor of size 1]


 0.9195
[torch.FloatTensor of size 1]


1.00000e-04 *
  5.2418
[torch.FloatTensor of size 1]


 0.7246
[torch.FloatTensor of size 1]


 0.8817
[torch.FloatTens

KeyboardInterrupt: 

In [ ]:
! wget https://raw.githubusercontent.com/pytorch/examples/master/imagenet/main.py

In [30]:
! python  main.py --pretrained -a alexnet --lr 0.01 --batch-size 64 --epochs 3 $data_path

=> using pre-trained model 'alexnet'
Epoch: [0][0/162]	Time 2.230 (2.230)	Data 0.383 (0.383)	Loss 2.6642 (2.6642)	Prec@1 17.188 (17.188)	Prec@5 45.312 (45.312)
Epoch: [0][10/162]	Time 1.477 (1.560)	Data 0.000 (0.035)	Loss 1.4033 (1.9735)	Prec@1 81.250 (58.097)	Prec@5 98.438 (84.801)
Epoch: [0][20/162]	Time 1.466 (1.514)	Data 0.000 (0.019)	Loss 0.8170 (1.5640)	Prec@1 82.812 (69.345)	Prec@5 98.438 (90.253)
Epoch: [0][30/162]	Time 1.490 (1.504)	Data 0.000 (0.013)	Loss 0.1957 (1.2364)	Prec@1 92.188 (74.647)	Prec@5 100.000 (93.246)
Epoch: [0][40/162]	Time 1.467 (1.503)	Data 0.000 (0.010)	Loss 0.8900 (1.1346)	Prec@1 89.062 (77.896)	Prec@5 98.438 (94.588)
Epoch: [0][50/162]	Time 1.463 (1.499)	Data 0.000 (0.008)	Loss 0.4776 (1.0806)	Prec@1 93.750 (80.270)	Prec@5 100.000 (95.558)
Epoch: [0][60/162]	Time 1.449 (1.495)	Data 0.000 (0.007)	Loss 0.6770 (1.0502)	Prec@1 90.625 (81.634)	Prec@5 100.000 (96.260)
Epoch: [0][70/162]	Time 1.487 (1.490)	Data 0.000 (0.006)	Loss 1.1414 (1.0304)	Prec@1 87.500 (

In [31]:
!ls -ltr model_best.pth.tar

-rw-r--r--  1 allaingu  1202659905  11185717 15 Jan 09:25 model_best.pth.tar


In [32]:
! python  main.py -a alexnet --pretrained --batch-size 64 --epochs 1 --evaluate --resume model_best.pth.tar $data_path

=> using pre-trained model 'alexnet'
=> loading checkpoint 'model_best.pth.tar'
=> loaded checkpoint 'model_best.pth.tar' (epoch 3)
Test: [0/3]	Time 1.957 (1.957)	Loss 0.6364 (0.6364)	Prec@1 92.188 (92.188)	Prec@5 100.000 (100.000)
 * Prec@1 44.865 Prec@5 88.649


# with non-linearities

In [35]:
! python  main.py --pretrained -a alexnet --lr 0.01 --batch-size 64 --epochs 3 $data_path

=> using pre-trained model 'alexnet'
Epoch: [0][0/162]	Time 2.432 (2.432)	Data 0.374 (0.374)	Loss 2.6699 (2.6699)	Prec@1 3.125 (3.125)	Prec@5 23.438 (23.438)
Epoch: [0][10/162]	Time 1.648 (1.742)	Data 0.000 (0.034)	Loss 0.9338 (1.5843)	Prec@1 65.625 (53.409)	Prec@5 96.875 (77.557)
Epoch: [0][20/162]	Time 1.708 (1.707)	Data 0.000 (0.018)	Loss 1.3779 (1.2944)	Prec@1 67.188 (62.054)	Prec@5 98.438 (86.086)
Epoch: [0][30/162]	Time 1.645 (1.692)	Data 0.000 (0.012)	Loss 0.7119 (1.1375)	Prec@1 76.562 (65.171)	Prec@5 96.875 (89.819)
Epoch: [0][40/162]	Time 1.638 (1.682)	Data 0.000 (0.009)	Loss 0.4872 (1.0014)	Prec@1 76.562 (68.941)	Prec@5 98.438 (91.959)
Epoch: [0][50/162]	Time 1.690 (1.680)	Data 0.000 (0.008)	Loss 0.3597 (0.8977)	Prec@1 87.500 (72.151)	Prec@5 100.000 (93.321)
Epoch: [0][60/162]	Time 1.652 (1.677)	Data 0.000 (0.006)	Loss 0.3216 (0.8341)	Prec@1 87.500 (73.924)	Prec@5 100.000 (94.314)
Epoch: [0][70/162]	Time 1.659 (1.671)	Data 0.000 (0.006)	Loss 0.3172 (0.7822)	Prec@1 90.625 (75.

In [36]:
! python  main.py -a alexnet --pretrained --batch-size 64 --epochs 1 --evaluate --resume model_best.pth.tar $data_path

=> using pre-trained model 'alexnet'
=> loading checkpoint 'model_best.pth.tar'
=> loaded checkpoint 'model_best.pth.tar' (epoch 3)
Test: [0/3]	Time 1.905 (1.905)	Loss 0.2425 (0.2425)	Prec@1 92.188 (92.188)	Prec@5 100.000 (100.000)
 * Prec@1 45.405 Prec@5 85.405


In [41]:
!ls -ltr  ../../chess-id/src/chessid/non_lin_model_best.pth.tar

-rw-r--r--  1 allaingu  1202659905  112756735 15 Jan 09:57 ../../chess-id/src/chessid/non_lin_model_best.pth.tar


In [42]:
best = torch.load('../../chess-id/src/chessid/non_lin_model_best.pth.tar')

In [46]:
torch.save(best['state_dict'], '../../chess-id/src/chessid/non_lin_model_only_best.pth.tar')
!ls -ltr ../../chess-id/src/chessid/non_lin_model_only_best.pth.tar

-rw-r--r--  1 allaingu  1202659905  61318373 15 Jan 11:39 ../../chess-id/src/chessid/non_lin_model_only_best.pth.tar


In [49]:
ImageClassifier(num_classes=13, alexnet_model=models.alexnet(pretrained=True)).load_state_dict(torch.load('../../chess-id/src/chessid/non_lin_model_only_best.pth.tar'))